In [ ]:
!pip -q install pandas numpy scikit-learn geopy haversine folium geopandas shapely pyproj plotly openpyxl

In [ ]:
# (A) Subir archivos manualmente
from google.colab import files
import io
import pandas as pd

print("Sube el archivo de Tiendas Empresa1 (Excel/CSV)")
uploaded_alq = files.upload()
alq_name = list(uploaded_alq.keys())[0]
if alq_name.lower().endswith(".csv"):
    df_alq = pd.read_csv(io.BytesIO(uploaded_alq[alq_name]))
else:
    df_alq = pd.read_excel(io.BytesIO(uploaded_alq[alq_name]))

print("Sube el archivo de Tiendas Aliados (Excel/CSV)")
uploaded_ali = files.upload()
ali_name = list(uploaded_ali.keys())[0]
if ali_name.lower().endswith(".csv"):
    df_ali = pd.read_csv(io.BytesIO(uploaded_ali[ali_name]))
else:
    df_ali = pd.read_excel(io.BytesIO(uploaded_ali[ali_name]))

print(df_alq.shape, df_ali.shape)
df_alq.head(3), df_ali.head(3)


Sube el archivo de Tiendas Alquería (Excel/CSV)


Saving ALQUERIA.xlsx to ALQUERIA (1).xlsx
Sube el archivo de Tiendas Aliados (Excel/CSV)


Saving ALIADOS.xlsx to ALIADOS (1).xlsx
(11632, 23) (4843, 23)


(  EMPRESA     CODIGO      Columna3 REGIONAL1 REGIONAL CANAL_SUPE  DISTRITO  \
 0      GA  451767000  451767  -000       RBO      NTE         PV         8   
 1      GA  752446000  752446  -000       RBO      NTE         PV         1   
 2      GA  504612000  504612  -000       RBO      NTE         PV         1   
 
   RUTA_ENTREGA  ZONA                        NOMBRE  ...    CEDULA TIPOLOGIA  \
 0        B0140     9     MILLERLEY LINARES SANCHEZ  ...  17267276        CL   
 1        B0153    16  MARTINEZ JOYA WILLIAM JAVIER  ...  74187956        TV   
 2        B0153    16            TRIANA RAYO ALVARO  ...    229846        TR   
 
       BARRIO  LONGITUD LATITUD      CIUDAD     DIA  TOMA DE PEDIDO  \
 0     CAJICÁ  -74.1666  4.6203  BOGOTÁ, DC      10          SABADO   
 1    MIRADOR  -74.1463  4.7113  BOGOTÁ, DC  101010          MAJUSA   
 2  MIRADOR 2  -74.1455  4.7116  BOGOTÁ, DC  101010          MAJUSA   
 
   DIA_ENTREGA  ENTREGA  
 0     1000000    LUNES  
 1     1010100   LUMIV

In [ ]:
import numpy as np

def _to_float(x):
    # Convierte "4,6123" -> 4.6123 y maneja nulos/errores
    if pd.isna(x):
        return np.nan
    if isinstance(x, (float, int)):
        return float(x)
    x = str(x).strip().replace(",", ".")
    try:
        return float(x)
    except:
        return np.nan

def clean_df(df, empresa_col='EMPRESA', nombre_col='NOMBRE', ciudad_col='CIUDAD',
             region_col='REGIONAL', lat_col='LATITUD', lon_col='LONGITUD'):
    # Estandariza texto
    for col in [empresa_col, nombre_col, ciudad_col, region_col]:
        if col in df.columns:
            df[col] = (df[col].astype(str)
                       .str.upper()
                       .str.strip()
                       .str.replace(r'\s+', ' ', regex=True))
    # Coordenadas a float
    df[lat_col] = df[lat_col].apply(_to_float)
    df[lon_col] = df[lon_col].apply(_to_float)
    # Valida rango Colombia
    df['COORD_VALIDA'] = (
        df[lat_col].between(-5, 13, inclusive='both') &
        df[lon_col].between(-81, -66, inclusive='both')
    )
    # Marca faltantes
    df['COORD_FALTANTE'] = df[lat_col].isna() | df[lon_col].isna()
    # ID único
    df['ID_TIENDA'] = (
        df.get(empresa_col, 'NA') + ' | ' +
        df.get(ciudad_col, 'NA') + ' | ' +
        df.get(nombre_col, 'NA') + ' | ' +
        df[lat_col].astype(str) + ' | ' +
        df[lon_col].astype(str)
    )
    # Elimina duplicados exactos de ID (conserva primero)
    df = df.drop_duplicates(subset=['ID_TIENDA']).copy()
    return df

df_alq = clean_df(df_alq)
df_ali = clean_df(df_ali)

# Filtra solo coordenadas válidas (mantén una copia si necesitas todo)
df_alq_valid = df_alq[(~df_alq['COORD_FALTANTE']) & (df_alq['COORD_VALIDA'])].copy()
df_ali_valid = df_ali[(~df_ali['COORD_FALTANTE']) & (df_ali['COORD_VALIDA'])].copy()

print({
    "Empresa1 registros": len(df_alq),
    "Empresa1 válidos": len(df_alq_valid),
    "Aliados registros": len(df_ali),
    "Aliados válidos": len(df_ali_valid)
})


{'Alquería registros': 11624, 'Alquería válidos': 11624, 'Aliados registros': 4843, 'Aliados válidos': 4843}


In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import BallTree
from geopy.distance import geodesic
import os

# ---------- Parámetros ----------
RADIO_M = 500
EARTH_RADIUS_M = 6371000.0
radio_rad = RADIO_M / EARTH_RADIUS_M
BATCH_SIZE = 4000                      # ajusta según RAM (2k–10k suele ir bien)
SALIDA_CSV = 'Cruces_500m_incremental.csv'
ESCRIBIR_EXCEL_FINAL = True            # al final convertimos a Excel

# ---------- Columnas mínimas ----------
cols_keep = ['ID_TIENDA','EMPRESA','NOMBRE','CIUDAD','REGIONAL','LATITUD','LONGITUD']
alq_slim = df_alq_valid[cols_keep].copy()
ali_slim = df_ali_valid[cols_keep].copy()

# Limpia salida si existe de antes
if os.path.exists(SALIDA_CSV):
    os.remove(SALIDA_CSV)

# Procesa ciudad por ciudad para reducir combinaciones
ciudades_comunes = np.intersect1d(alq_slim['CIUDAD'].unique(), ali_slim['CIUDAD'].unique())
print(f"Ciudades en común: {len(ciudades_comunes)}")

header_escrito = False
total_pairs = 0

for ciudad in ciudades_comunes:
    alq_city = alq_slim[alq_slim['CIUDAD'] == ciudad].reset_index(drop=True)
    ali_city = ali_slim[ali_slim['CIUDAD'] == ciudad].reset_index(drop=True)
    if alq_city.empty or ali_city.empty:
        continue

    # Árbol solo para aliados de la ciudad
    ali_coords_rad = np.radians(ali_city[['LATITUD','LONGITUD']].to_numpy())
    ali_tree = BallTree(ali_coords_rad, metric='haversine')

    # Lotes de Empresa1 en esa ciudad
    for start in range(0, len(alq_city), BATCH_SIZE):
        end = min(start + BATCH_SIZE, len(alq_city))
        alq_batch = alq_city.iloc[start:end].reset_index(drop=True)
        alq_coords_rad = np.radians(alq_batch[['LATITUD','LONGITUD']].to_numpy())

        # Vecinos dentro del radio
        idxs_list, dists_list = ali_tree.query_radius(alq_coords_rad, r=radio_rad, return_distance=True)

        # Construye un DataFrame pequeño por lote y escribe a disco
        rows = []
        for i_alq_batch, (idxs, dists) in enumerate(zip(idxs_list, dists_list)):
            if len(idxs) == 0:
                continue
            alq_row = alq_batch.iloc[i_alq_batch]
            for j_ali, dist_rad in zip(idxs, dists):
                ali_row = ali_city.iloc[j_ali]
                dist_m = float(dist_rad * EARTH_RADIUS_M)
                rows.append({
                    'ID_EMPRESA1': alq_row['ID_TIENDA'],
                    'NOMBRE_EMPRESA1': alq_row['NOMBRE'],
                    'CIUDAD_EMPRESA1': alq_row['CIUDAD'],
                    'REGION_EMPRESA1': alq_row['REGIONAL'],
                    'LAT_ALQ': alq_row['LATITUD'],
                    'LON_ALQ': alq_row['LONGITUD'],
                    'ID_ALIADO': ali_row['ID_TIENDA'],
                    'NOMBRE_ALIADO': ali_row['NOMBRE'],
                    'CIUDAD_ALIADO': ali_row['CIUDAD'],
                    'REGION_ALIADO': ali_row['REGIONAL'],
                    'LAT_ALI': ali_row['LATITUD'],
                    'LON_ALI': ali_row['LONGITUD'],
                    'DISTANCIA_MTS': round(dist_m, 2)
                })

        if rows:
            total_pairs += len(rows)
            df_out = pd.DataFrame(rows)
            df_out['CERCANIA'] = (df_out['DISTANCIA_MTS'] <= RADIO_M).astype(int)
            df_out.to_csv(SALIDA_CSV, mode='a', index=False, header=not header_escrito, encoding='utf-8')
            header_escrito = True

print(f"Pares totales dentro de {RADIO_M} m: {total_pairs}")

# Carga resultado incremental para seguir con los análisis (si hay datos)
if total_pairs > 0:
    resultado_500m = pd.read_csv(SALIDA_CSV)
else:
    resultado_500m = pd.DataFrame(columns=[
        'ID_EMPRESA1','NOMBRE_EMPRESA1','CIUDAD_EMPRESA1','REGION_EMPRESA1','LAT_ALQ','LON_ALQ',
        'ID_ALIADO','NOMBRE_ALIADO','CIUDAD_ALIADO','REGION_ALIADO','LAT_ALI','LON_ALI',
        'DISTANCIA_MTS','CERCANIA'
    ])

# (Opcional) exportar a Excel al final
if ESCRIBIR_EXCEL_FINAL and total_pairs > 0:
    with pd.ExcelWriter('Cruce_Distancias_Tiendas.xlsx', engine='openpyxl') as writer:
        resultado_500m.to_excel(writer, sheet_name='Cruces_500m', index=False)


Ciudades en común: 1
Pares totales dentro de 500 m: 557153


In [ ]:
from google.colab import files
files.download('Cruce_Distancias_Tiendas.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# ===============================
# MAPA FOLIUM: PUNTOS + DISTANCIAS
# ===============================

import folium
import random
import pandas as pd

# -------------------------------
# CONFIGURACIÓN (ajusta si deseas)
# -------------------------------
MAX_PUNTOS = 4000     # puntos totales visibles
MAX_LINEAS = 400      # líneas de distancia
ZOOM = 12             # zoom de ciudad

# -------------------------------
# VALIDAR COORDENADAS
# -------------------------------
df_alq_valid = df_alq.dropna(subset=['LATITUD','LONGITUD'])
df_ali_valid = df_ali.dropna(subset=['LATITUD','LONGITUD'])

# -------------------------------
# CENTRO DEL MAPA (promedio)
# -------------------------------
center_lat = df_alq_valid['LATITUD'].mean()
center_lon = df_alq_valid['LONGITUD'].mean()

m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=ZOOM,
    control_scale=True
)

# -------------------------------
# CAPAS
# -------------------------------
fg_alq = folium.FeatureGroup(name="Tiendas Empresa1")
fg_ali = folium.FeatureGroup(name="Tiendas Aliados")
fg_lines = folium.FeatureGroup(name="Distancias")

# -------------------------------
# SAMPLE SEGURO DE PUNTOS
# -------------------------------
alq_pts = (
    df_alq_valid[['LATITUD','LONGITUD']]
    .drop_duplicates()
)
ali_pts = (
    df_ali_valid[['LATITUD','LONGITUD']]
    .drop_duplicates()
)

alq_n = min(MAX_PUNTOS // 2, len(alq_pts))
ali_n = min(MAX_PUNTOS // 2, len(ali_pts))

alq_pts = alq_pts.sample(alq_n, random_state=42)
ali_pts = ali_pts.sample(ali_n, random_state=42)

# -------------------------------
# PUNTOS EMPRESA1 (AZUL)
# -------------------------------
for _, r in alq_pts.iterrows():
    folium.CircleMarker(
        location=[r['LATITUD'], r['LONGITUD']],
        radius=4,
        color='blue',
        fill=True,
        fill_opacity=0.7
    ).add_to(fg_alq)

# -------------------------------
# PUNTOS ALIADOS (ROJO)
# -------------------------------
for _, r in ali_pts.iterrows():
    folium.CircleMarker(
        location=[r['LATITUD'], r['LONGITUD']],
        radius=4,
        color='red',
        fill=True,
        fill_opacity=0.7
    ).add_to(fg_ali)

# -------------------------------
# LÍNEAS DE DISTANCIA (ALEATORIAS)
# -------------------------------
line_count = min(MAX_LINEAS, len(alq_pts) * len(ali_pts))

for _ in range(line_count):
    a = alq_pts.sample(1).iloc[0]
    b = ali_pts.sample(1).iloc[0]

    folium.PolyLine(
        locations=[
            [a['LATITUD'], a['LONGITUD']],
            [b['LATITUD'], b['LONGITUD']]
        ],
        color='gray',
        weight=1,
        opacity=0.4
    ).add_to(fg_lines)

# -------------------------------
# AGREGAR CAPAS AL MAPA
# -------------------------------
fg_alq.add_to(m)
fg_ali.add_to(m)
fg_lines.add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

# -------------------------------
# MOSTRAR MAPA
# -------------------------------
m
